### Duck typing

In [ ]:
from hagis import Layer

layer = Layer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/USA/MapServer/0")

for city in layer.query():
    print(city.areaname)


### Querying with a type argument

In [ ]:
from hagis import Layer, Point

class City:
    objectid: int
    areaname: str
    pop2000: int
    shape: Point

layer = Layer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/USA/MapServer/0", City)

for city in layer.query("pop2000 > 1000000", orderByFields="pop2000 DESC", outSR=102100):
    print(city.areaname, city.pop2000, city.shape.x, city.shape.y)


### Mapping to a dataclass

In [ ]:
from hagis import Layer, Polyline
from dataclasses import dataclass

@dataclass(frozen=True)
class Highway:
    objectid: int
    route: str
    shape: Polyline

layer = Layer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/USA/MapServer/1", Highway)

for highway in layer.query():
    print(highway.route)


### Mapping to Esri geometry

In [ ]:
from hagis import Layer
from arcgis.geometry import Polygon

class State:
    state_name: str
    pop2000: int
    shape: Polygon

layer = Layer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/USA/MapServer/2", State)

for state in layer.query(record_count=4, orderByFields="pop2000 DESC"):
    print(state.state_name, state.pop2000, state.shape.centroid)
    display(state.shape.buffer(0.2))


### Custom property to field mapping

In [ ]:
from hagis import Layer, Polyline

class Highway:
    objectid: int
    name: str  # This is actually called "route" in the table.
    shape: Polyline

custom_mapping = {
    "name": "route",
    "type": "type" # This is not even defined in the Highway class (i.e. customer-specific field).
}

layer = Layer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/USA/MapServer/1", Highway, **custom_mapping)

for highway in layer.query():
    print(highway.name, getattr(highway, "type")) # Can be accessed like highway.type (though it doesn't make sense to do so).


### Coded value domain

In [ ]:
from enum import Enum
from typing import Optional
from uuid import UUID
from hagis import Layer, Point

class Factype(Enum):
    Public_Safety_Resource = 1
    Pet_Collection_Shelter = 2
    Shelter = 3

class Opsstatus(Enum):
    Open = "Open"
    Closed = "Closed"
    Unknown = "Unknown"

class Facility:
    objectid: int
    facilityid: Optional[str]
    factype: Optional[Factype]
    opsstatus: Optional[Opsstatus]
    globalid: UUID
    shape: Point

layer = Layer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/EmergencyFacilities/FeatureServer/0", Facility)

for facility in layer.query():
    if facility.factype == Factype.Public_Safety_Resource:
        print(facility.__dict__)


### Lazily-evaluated sequence

In [ ]:
from hagis import Layer
from datetime import datetime

class Rain:
    objectid: int
    site_no: str
    date_time: datetime
    rainfall_inch: float

layer = Layer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/SpatioTemporalAggregation/RainfallTimeSeriesDataIllinois/MapServer/7", Rain)

print(f"This layer has {layer.count()} records!")

# Keep querying.
for rain in layer.query(record_count=12345):
    print(rain.__dict__)


### Insert

In [ ]:
from hagis import Layer, Point

class Ambulance:
    objectid: int
    unitname: str
    type: int
    speed: float
    shape: Point

layer = Layer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/RedlandsEmergencyVehicles/FeatureServer/0", Ambulance)

ambulance = Ambulance()
ambulance.unitname = "Created by https://pypi.org/project/hagis/"
ambulance.type = 1
ambulance.speed = 12.34567
ambulance.shape = Point()
ambulance.shape.x, ambulance.shape.y = -120, 50

# Returns the newly created object ids.
oid = layer.insert([ambulance])[0]

layer.find(oid).__dict__


### Update

In [ ]:
from hagis import Layer

class Ambulance:
    objectid: int
    unitname: str

layer = Layer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/RedlandsEmergencyVehicles/FeatureServer/0", Ambulance)

where_clause = "unitname LIKE '%hagis%'"

# Update requires materializing records first.
ambulances = list(layer.query(where_clause))

for ambulance in ambulances:
    ambulance.unitname = "Updated by https://pypi.org/project/hagis/"

# Send the list back to the server.
layer.update(ambulances)

for ambulance in layer.query(where_clause):
    print(ambulance.__dict__)


### Delete

In [ ]:
from hagis import Layer

layer = Layer("https://sampleserver6.arcgisonline.com/arcgis/rest/services/RedlandsEmergencyVehicles/FeatureServer/0")

where_clause = "unitname LIKE '%hagis%'"

print("Before", layer.count(where_clause))

# Delete requires the where clause.
layer.delete(where_clause)

print("After", layer.count(where_clause))


### Generic operations

In [ ]:

from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Iterator, Optional, Type, TypeVar
from hagis import Layer, Point, Polyline, Polygon

URL = "https://sampleserver6.arcgisonline.com/arcgis/rest/services/Wildfire/FeatureServer"

@dataclass(frozen=True)
class Tracked:
    objectid: int
    created_user: str
    created_date: datetime
    last_edited_user: Optional[str]
    last_edited_date: Optional[datetime]

@dataclass(frozen=True)
class ResponsePoint(Tracked):
    rotation: Optional[int]
    description: Optional[str]
    shape: Point

@dataclass(frozen=True)
class ResponseLine(Tracked):
    symbolid: Optional[int]
    timestamp: Optional[datetime]
    description: Optional[str]
    shape__length: float
    shape: Polyline

@dataclass(frozen=True)
class ResponsePolygon(Tracked):
    objectid: int
    symbolid: Optional[int]
    description: Optional[str]
    shape__area: float
    shape: Polygon

T = TypeVar("T", bound=Tracked)

def get_recent_responses(model: Type[T], layer_id: int, hours: int) -> Iterator[T]:
    layer = Layer(f"{URL}/{layer_id}", model)
    time = datetime.utcnow() + timedelta(hours=-hours)
    for response in layer.query(lambda x: x.last_edited_date is not None and x.last_edited_date > time):
        yield response

last_n_hours = 4

print(f"Response points edited in the last {last_n_hours} hour(s)")
for r in get_recent_responses(ResponsePoint, 0, last_n_hours):
    print(r.last_edited_date, r.rotation)

print(f"Response lines edited in the last {last_n_hours} hour(s)")
for r in get_recent_responses(ResponseLine, 1, last_n_hours):
    print(r.last_edited_date, r.timestamp)

print(f"Response polygons edited in the last {last_n_hours} hour(s)")
for r in get_recent_responses(ResponsePolygon, 2, last_n_hours):
    print(r.last_edited_date, r.symbolid)


### Authentication

In [ ]:
from hagis import Layer
from getpass import getpass

layer = Layer("https://services8.arcgis.com/hlI3WvZMbprlF0sO/ArcGIS/rest/services/Redlands_65985/FeatureServer/0")

# Self-renewing token.
layer.set_token_generator("jshirota", getpass(), "https://myserver/mywebapp")

# Static token.
# layer.set_token("token123")

for ambulance in layer.query():
    print(ambulance)
